# 配方设计计算器

## 功能说明

本脚本根据历史酿造数据的效率统计值，计算达到目标麦汁所需的原料用量和工艺参数。

### 计算步骤

1. **计算麦芽重量**: 根据目标麦汁体积、比重、熬煮后效率和理论浸出率
2. **计算糖化用水量**: 根据麦芽重量和水料比
3. **估计糖化后比重**: 根据糖化后提取率，用于过程监控
4. **计算洗糟水用量**: 根据谷物吸收率和熬煮蒸发量
5. **估计洗糟后比重**: 根据洗糟后提取率，用于过程监控

### 计算公式

$$M_{\text{act}}(V, SG) = 2.5 \times (SG - 1) \times V \times SG$$

$$\text{Efficiency} = \frac{M_{\text{Actual Extract}}}{M_{\text{Potential Extract}}} \times 100\%$$

$$M_{\text{Potential Extract}} = \text{麦芽重量} \times \text{理论浸出率}$$

$$\text{Grain Absorption Rate} = \frac{\text{糖化水} + \text{洗糟水} - \text{熬煮前体积}}{\text{麦芽重量}}$$

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path

print("📊 配方设计计算器")
print("="*60)

📊 配方设计计算器


## 1. 加载历史效率数据

In [13]:
# 从CSV文件加载历史效率数据
metrics_file = 'brewing_metrics_summary.csv'
metrics_df = pd.read_csv(metrics_file)

# 提取关键指标
mash_extraction_efficiency = metrics_df['mash_extraction_efficiency'].values[0]
sparge_extraction_efficiency = metrics_df['sparge_extraction_efficiency'].values[0]
grain_absorption_rate = metrics_df['grain_absorption_rate'].values[0]
post_boil_efficiency = metrics_df['post_boil_efficiency'].values[0]
boil_off_volume = metrics_df['boil_off_volume'].values[0]

print("✅ 历史效率数据加载成功！")
print("-"*60)
print(f"糖化后提取率: {mash_extraction_efficiency:.2f}%")
print(f"洗糟后提取率: {sparge_extraction_efficiency:.2f}%")
print(f"谷物吸收率: {grain_absorption_rate:.3f} L/kg")
print(f"熬煮后效率: {post_boil_efficiency:.2f}%")
print(f"熬煮蒸发量: {boil_off_volume:.2f} L")
print("="*60)

✅ 历史效率数据加载成功！
------------------------------------------------------------
糖化后提取率: 70.77%
洗糟后提取率: 79.47%
谷物吸收率: 0.862 L/kg
熬煮后效率: 68.70%
熬煮蒸发量: 1.70 L


## 2. 设置酿造目标

In [14]:
# ==================== 酿造目标参数 ====================
# 目标麦汁参数
target_post_boil_volume = 22.0  # 煮沸后麦汁体积 (L)
target_post_boil_sg = 1.055     # 煮沸后麦汁比重

# 工艺参数
potential_extract_rate = 0.78   # 理论浸出率 (综合值)
water_grain_ratio = 3.2         # 水料比

print("🎯 酿造目标参数")
print("-"*60)
print(f"煮沸后麦汁体积: {target_post_boil_volume} L")
print(f"煮沸后麦汁比重: {target_post_boil_sg}")
print(f"理论浸出率: {potential_extract_rate*100}%")
print(f"水料比: {water_grain_ratio}")
print("="*60)

🎯 酿造目标参数
------------------------------------------------------------
煮沸后麦汁体积: 22.0 L
煮沸后麦汁比重: 1.055
理论浸出率: 78.0%
水料比: 3.2


## 3. 定义计算函数

In [15]:
def calculate_actual_extract(volume_l, sg):
    """
    计算麦汁中的实际含糖量
    
    参数:
        volume_l: 麦汁体积 (L)
        sg: 麦汁比重
    
    返回:
        实际含糖量 (kg)
    """
    return 2.5 * (sg - 1.0) * volume_l * sg

def calculate_malt_weight(target_volume, target_sg, efficiency, potential_rate):
    """
    计算所需麦芽重量
    
    参数:
        target_volume: 目标麦汁体积 (L)
        target_sg: 目标麦汁比重
        efficiency: 效率 (%)
        potential_rate: 理论浸出率
    
    返回:
        麦芽重量 (kg)
    """
    actual_extract = calculate_actual_extract(target_volume, target_sg)
    potential_extract = actual_extract / (efficiency / 100)
    malt_weight = potential_extract / potential_rate
    return malt_weight

def estimate_sg_from_sugar(volume_l, sugar_mass):
    """
    根据糖量估计比重（基于糖量守恒）
    
    参数:
        volume_l: 麦汁体积 (L)
        sugar_mass: 糖的质量 (kg)
    
    返回:
        估计的比重
    
    说明：
        从 sugar_mass = 2.5 * (sg - 1) * volume * sg 反解sg
        2.5 * volume * sg^2 - 2.5 * volume * sg - sugar_mass = 0
    """
    a = 2.5 * volume_l
    b = -2.5 * volume_l
    c = -sugar_mass
    
    # 求解二次方程: a*sg^2 + b*sg + c = 0
    discriminant = b**2 - 4*a*c
    sg = (-b + discriminant**0.5) / (2*a)
    
    return sg

print("✅ 计算函数定义完成！")

✅ 计算函数定义完成！


## 4. 配方计算

### 步骤1: 计算麦芽重量

In [16]:
# 计算所需麦芽重量
total_malt_weight = calculate_malt_weight(
    target_post_boil_volume,
    target_post_boil_sg,
    post_boil_efficiency,
    potential_extract_rate
)

print("📋 步骤1: 计算麦芽重量")
print("-"*60)
print(f"目标麦汁实际含糖量: {calculate_actual_extract(target_post_boil_volume, target_post_boil_sg):.3f} kg")
print(f"使用熬煮后效率: {post_boil_efficiency:.2f}%")
print(f"所需麦芽重量: {total_malt_weight:.3f} kg")
print("="*60)

📋 步骤1: 计算麦芽重量
------------------------------------------------------------
目标麦汁实际含糖量: 3.191 kg
使用熬煮后效率: 68.70%
所需麦芽重量: 5.956 kg


### 步骤2: 计算糖化用水量

In [17]:
# 计算糖化用水量
strike_water = total_malt_weight * water_grain_ratio

print("📋 步骤2: 计算糖化用水量")
print("-"*60)
print(f"麦芽重量: {total_malt_weight:.3f} kg")
print(f"水料比: {water_grain_ratio}")
print(f"糖化用水量: {strike_water:.2f} L")
print("="*60)

📋 步骤2: 计算糖化用水量
------------------------------------------------------------
麦芽重量: 5.956 kg
水料比: 3.2
糖化用水量: 19.06 L


### 步骤3: 估计糖化后比重（用于过程监控）

In [18]:
# 估计糖化后比重
# 首先计算糖化后的实际糖量
potential_extract_total = total_malt_weight * potential_extract_rate
sugar_mash = potential_extract_total * (mash_extraction_efficiency / 100)

# 假设糖化后的麦汁体积约等于糖化用水量（麦芽吸收部分水后的体积）
mash_volume = strike_water  # 简化假设

estimated_pre_sparge_sg = estimate_sg_from_sugar(mash_volume, sugar_mash)

print("📋 步骤3: 估计糖化后比重")
print("-"*60)
print(f"麦芽理论总糖量: {potential_extract_total:.3f} kg")
print(f"糖化后实际糖量: {sugar_mash:.3f} kg (使用糖化提取率 {mash_extraction_efficiency:.2f}%)")
print(f"糖化后估计体积: {mash_volume:.2f} L")
print(f"估计糖化后比重: {estimated_pre_sparge_sg:.4f}")
print(f"  (过程监控目标值)")
print("="*60)

📋 步骤3: 估计糖化后比重
------------------------------------------------------------
麦芽理论总糖量: 4.646 kg
糖化后实际糖量: 3.288 kg (使用糖化提取率 70.77%)
糖化后估计体积: 19.06 L
估计糖化后比重: 1.0648
  (过程监控目标值)


### 步骤4: 计算洗糟水用量

In [19]:
# 计算熬煮前体积
pre_boil_volume = target_post_boil_volume + boil_off_volume

# 根据谷物吸收率计算洗糟水用量
# grain_absorption_rate = (strike_water + sparge_water - pre_boil_volume) / total_malt_weight
# sparge_water = grain_absorption_rate * total_malt_weight + pre_boil_volume - strike_water
sparge_water = grain_absorption_rate * total_malt_weight + pre_boil_volume - strike_water

print("📋 步骤4: 计算洗糟水用量")
print("-"*60)
print(f"熬煮蒸发量: {boil_off_volume:.2f} L")
print(f"熬煮前体积: {pre_boil_volume:.2f} L")
print(f"谷物吸收率: {grain_absorption_rate:.3f} L/kg")
print(f"洗糟水用量: {sparge_water:.2f} L")
print("="*60)

📋 步骤4: 计算洗糟水用量
------------------------------------------------------------
熬煮蒸发量: 1.70 L
熬煮前体积: 23.70 L
谷物吸收率: 0.862 L/kg
洗糟水用量: 9.77 L


### 步骤5: 估计洗糟后比重（用于过程监控）

In [20]:
# 估计洗糟后比重
# 根据提取率公式反推：η = M_act(体积, 比重) / M_potential

# 已知量
total_water = strike_water + sparge_water  # 总用水量
potential_extract_total = total_malt_weight * potential_extract_rate  # 理论总糖量

# 根据洗糟后提取率计算洗糟后应该含有的糖量
sugar_pre_boil_by_efficiency = potential_extract_total * (sparge_extraction_efficiency / 100)

# 用糖量和总体积计算洗糟后比重
estimated_pre_boil_sg = estimate_sg_from_sugar(total_water, sugar_pre_boil_by_efficiency)

print("📋 步骤5: 估计洗糟后比重（基于提取率公式）")
print("-"*70)
print("\n【已知数据】")
print(f"  洗糟后提取率 η = {sparge_extraction_efficiency:.2f}% ({sparge_extraction_efficiency/100:.4f})")
print(f"  麦芽总重量 = {total_malt_weight:.3f} kg")
print(f"  理论浸出率 = {potential_extract_rate:.2f}")
print(f"  总水量 = {total_water:.2f} L (糖化水 {strike_water:.2f}L + 洗糟水 {sparge_water:.2f}L)")

print("\n【计算步骤】")
print(f"  ① 理论总糖量: {total_malt_weight:.3f} × {potential_extract_rate:.2f} = {potential_extract_total:.5f} kg")
print(f"  ② 洗糟后糖量: {potential_extract_total:.5f} × {sparge_extraction_efficiency/100:.4f} = {sugar_pre_boil_by_efficiency:.5f} kg")
print(f"  ③ 建立方程: {sugar_pre_boil_by_efficiency:.3f} = 2.5 × (SG - 1) × {total_water:.2f} × SG")

print("\n【求解二次方程】")
a = 2.5 * total_water
b = -2.5 * total_water
c = -sugar_pre_boil_by_efficiency
print(f"  方程: {a:.3f}×SG² + {b:.3f}×SG - {sugar_pre_boil_by_efficiency:.3f} = 0")

print("\n【计算结果】")
print(f"  ✅ 洗糟后比重 SG_sparge = {estimated_pre_boil_sg:.4f}")

print("\n【验证对比】")
# 计算煮沸后实际收获的糖量
sugar_post_boil_actual = calculate_actual_extract(target_post_boil_volume, target_post_boil_sg)
# 计算未回收的损失（管道、锅底残留等）
sugar_loss = sugar_pre_boil_by_efficiency - sugar_post_boil_actual
loss_percentage = (sugar_loss / sugar_pre_boil_by_efficiency) * 100

print(f"  煮沸后实际收获: {sugar_post_boil_actual:.3f} kg (体积{target_post_boil_volume}L, SG{target_post_boil_sg})")
print(f"  未回收损失: {sugar_loss:.3f} kg ({loss_percentage:.2f}%)")
print(f"  损失率 = {sparge_extraction_efficiency:.2f}% - {post_boil_efficiency:.2f}% = {sparge_extraction_efficiency - post_boil_efficiency:.2f}%")

print("\n💡 关键理解：")
print("  - 洗糟后提取率(79.47%)基于'总用水量'计算，反映总提取效果")
print("  - 煮沸后效率(68.70%)基于'实际收获体积'计算，反映最终收获")
print(f"  - 两者差异({sparge_extraction_efficiency - post_boil_efficiency:.2f}%)代表未回收损失")
print("="*70)

📋 步骤5: 估计洗糟后比重（基于提取率公式）
----------------------------------------------------------------------

【已知数据】
  洗糟后提取率 η = 79.47% (0.7947)
  麦芽总重量 = 5.956 kg
  理论浸出率 = 0.78
  总水量 = 28.83 L (糖化水 19.06L + 洗糟水 9.77L)

【计算步骤】
  ① 理论总糖量: 5.956 × 0.78 = 4.64572 kg
  ② 洗糟后糖量: 4.64572 × 0.7947 = 3.69186 kg
  ③ 建立方程: 3.692 = 2.5 × (SG - 1) × 28.83 × SG

【求解二次方程】
  方程: 72.079×SG² + -72.079×SG - 3.692 = 0

【计算结果】
  ✅ 洗糟后比重 SG_sparge = 1.0488

【验证对比】
  煮沸后实际收获: 3.191 kg (体积22.0L, SG1.055)
  未回收损失: 0.500 kg (13.56%)
  损失率 = 79.47% - 68.70% = 10.77%

💡 关键理解：
  - 洗糟后提取率(79.47%)基于'总用水量'计算，反映总提取效果
  - 煮沸后效率(68.70%)基于'实际收获体积'计算，反映最终收获
  - 两者差异(10.77%)代表未回收损失


## 5. 配方汇总

## 糖量平衡分析

In [21]:
# 创建配方汇总表格
recipe_summary = pd.DataFrame({
    '项目': [
        '========= 目标参数 ==========',
        '煮沸后体积 (L)',
        '煮沸后比重',
        '',
        '========= 原料用量 ==========',
        '麦芽总重量 (kg)',
        '理论浸出率 (%)',
        '',
        '========= 用水参数 ==========',
        '糖化用水量 (L)',
        '洗糟水用量 (L)',
        '总用水量 (L)',
        '水料比',
        '',
        '========= 过程监控 ==========',
        '糖化后比重估计值',
        '洗糟后体积 (L)',
        '洗糟后比重估计值',
        '熬煮蒸发量 (L)',
        ''
    ],
    '数值': [
        '',
        f'{target_post_boil_volume:.2f}',
        f'{target_post_boil_sg:.4f}',
        '',
        '',
        f'{total_malt_weight:.3f}',
        f'{potential_extract_rate*100:.0f}',
        '',
        '',
        f'{strike_water:.2f}',
        f'{sparge_water:.2f}',
        f'{total_water:.2f}',
        f'{water_grain_ratio:.1f}',
        '',
        '',
        f'{estimated_pre_sparge_sg:.4f}',
        f'{total_water:.2f}',
        f'{estimated_pre_boil_sg:.4f}',
        f'{boil_off_volume:.2f}',
        ''
    ]
})

print("\n" + "="*60)
print("📊 配方设计完成")
print("="*60)
display(recipe_summary)
print("="*60)


📊 配方设计完成


,项目,数值
0,========= 目标参数 ==========,
1,煮沸后体积 (L),22.00
2,煮沸后比重,1.0550
3,,
4,========= 原料用量 ==========,
5,麦芽总重量 (kg),5.956
6,理论浸出率 (%),78
7,,
8,========= 用水参数 ==========,
9,糖化用水量 (L),19.06


In [22]:
# 糖量平衡分析
# 计算各阶段的实际糖量
potential_sugar_total = total_malt_weight * potential_extract_rate  # 理论总糖量
sugar_mash = potential_sugar_total * (mash_extraction_efficiency / 100)  # 糖化后提取糖量
sugar_sparge = potential_sugar_total * (sparge_extraction_efficiency / 100)  # 洗糟后糖量
sugar_boil = calculate_actual_extract(target_post_boil_volume, target_post_boil_sg)  # 煮沸后收获糖量

sugar_balance = pd.DataFrame({
    '阶段': [
        '1. 理论总糖量',
        '2. 糖化后提取',
        '3. 洗糟后收集',
        '4. 煮沸后收获'
    ],
    '糖量 (kg)': [
        f'{potential_sugar_total:.3f}',
        f'{sugar_mash:.3f}',
        f'{sugar_sparge:.3f}',
        f'{sugar_boil:.3f}'
    ],
    '累计提取率': [
        '100% (理论)',
        f'{mash_extraction_efficiency:.2f}%',
        f'{sparge_extraction_efficiency:.2f}%',
        f'{post_boil_efficiency:.2f}%'
    ],
    '体积 (L)': [
        '-',
        f'{strike_water:.2f}',
        f'{total_water:.2f}',
        f'{target_post_boil_volume:.2f}'
    ],
    '比重': [
        '-',
        f'{estimated_pre_sparge_sg:.4f}',
        f'{estimated_pre_boil_sg:.4f}',
        f'{target_post_boil_sg:.4f}'
    ]
})

print("\n" + "="*110)
print("📊 糖量平衡与浓度变化分析")
print("="*110)
display(sugar_balance.style.set_properties(**{'text-align': 'center'}))
print()
print("🔍 关键规律说明：")
print("-" * 110)
print("【糖量变化】")
print(f"  ①→② 糖化提取: {potential_sugar_total:.3f} → {sugar_mash:.3f} kg ({mash_extraction_efficiency:.2f}%)")
print(f"  ②→③ 洗糟提取: {sugar_mash:.3f} → {sugar_sparge:.3f} kg (增加 {sugar_sparge-sugar_mash:.3f} kg)")
print(f"  ③→④ 煮沸损失: {sugar_sparge:.3f} → {sugar_boil:.3f} kg (损失 {sugar_sparge-sugar_boil:.3f} kg, {(sugar_sparge-sugar_boil)/sugar_sparge*100:.2f}%)")
print()
print("【体积与浓度变化】")
print(f"  阶段② - 糖化后: 体积 {strike_water:.2f}L, 比重 {estimated_pre_sparge_sg:.4f}")
print(f"  阶段③ - 洗糟后: 体积 {total_water:.2f}L (+{total_water-strike_water:.2f}L), 比重 {estimated_pre_boil_sg:.4f}")
print(f"  阶段④ - 煮沸后: 体积 {target_post_boil_volume:.2f}L (-{total_water-target_post_boil_volume:.2f}L), 比重 {target_post_boil_sg:.4f}")
print()
print("✅ 物理规律验证：")
print(f"  洗糟后体积({total_water:.2f}L) > 煮沸后体积({target_post_boil_volume:.2f}L)")
print(f"  洗糟后比重({estimated_pre_boil_sg:.4f}) < 煮沸后比重({target_post_boil_sg:.4f})")
print(f"  ✓ 体积越大，浓度越低，符合物理规律！")
print("="*110)


📊 糖量平衡与浓度变化分析


,阶段,糖量 (kg),累计提取率,体积 (L),比重
0,1. 理论总糖量,4.646,100% (理论),-,-
1,2. 糖化后提取,3.288,70.77%,19.06,1.0648
2,3. 洗糟后收集,3.692,79.47%,28.83,1.0488
3,4. 煮沸后收获,3.191,68.70%,22.00,1.0550



🔍 关键规律说明：
--------------------------------------------------------------------------------------------------------------
【糖量变化】
  ①→② 糖化提取: 4.646 → 3.288 kg (70.77%)
  ②→③ 洗糟提取: 3.288 → 3.692 kg (增加 0.404 kg)
  ③→④ 煮沸损失: 3.692 → 3.191 kg (损失 0.500 kg, 13.56%)

【体积与浓度变化】
  阶段② - 糖化后: 体积 19.06L, 比重 1.0648
  阶段③ - 洗糟后: 体积 28.83L (+9.77L), 比重 1.0488
  阶段④ - 煮沸后: 体积 22.00L (-6.83L), 比重 1.0550

✅ 物理规律验证：
  洗糟后体积(28.83L) > 煮沸后体积(22.00L)
  洗糟后比重(1.0488) < 煮沸后比重(1.0550)
  ✓ 体积越大，浓度越低，符合物理规律！
